In [ ]:
# importerer generelle moduler

import os
import glob
import torch
import torchvision.transforms as T
import numpy as np
import utils
import unet
from pathlib import Path
from tqdm import tqdm
import rasterio
from rasterio.plot import show

In [ ]:
# setter overordnede variabler

sensor = "s1s2"
classes = ['background', 'built-up', 'roads']
n_classes = len(classes)
chip_size = 512
buffer = 20
name = '3cls_rnd'

# sette navn på forsøk
experiment_name = f"{sensor}_{chip_size}_{name}_01"
print(experiment_name)

year = 2022

if sensor == "s1":
    n_channels = 2
    sensor_name = "Sentinel-1"
    scenes = glob.glob(os.path.join(f"/filserver/user/imagery/{sensor_name}/{year}/", '*_aligned.tiff'))
elif sensor == "s2":
    n_channels = 12
    sensor_name = "Sentinel-2"
    scenes = glob.glob(os.path.join(f"/filserver/user/imagery/{sensor_name}/{year}/", '*_aligned_12b.tiff'))
elif sensor == "s1s2":
    n_channels = 14
    sensor_name = "Sentinel-1_Sentinel-2"
    scenes = glob.glob(os.path.join(f"/filserver/user/imagery/{sensor_name}/{year}/", '*_aligned_sar.tiff'))


archive = f"/filserver/user/imagery/{sensor_name}/{year}/"
print(f"antall {sensor_name} scener i {year}: {len(scenes)}")

In [ ]:
# definer et nett
net = unet.UNet(
    encChannels=[n_channels,64,128,256,512],
    decChannels=[512,256,128,64],
    nbClasses=n_classes,
    outSize=(chip_size,chip_size))

if torch.cuda.is_available():
    net.load_state_dict(torch.load(f"../models/{experiment_name}/models/unet_best.pt",map_location="cuda"))
    net = net.to("cuda")
else:
    net.load_state_dict(torch.load(f"../models/{experiment_name}/models/unet_best.pt",map_location="cpu"))

In [ ]:
for i, scene in enumerate(scenes):
    image_name = Path(scene).stem
    print(i+1,'/',len(scenes))
    print(f"loading image {image_name}")
    img = rasterio.open(scene)
    # if sensor == "s1":
    #     r = utils.pct_clip(img.read(2))
    #     g = utils.pct_clip(img.read(1))
    #     b = utils.pct_clip(img.read(2)) / utils.pct_clip(img.read(1))
    #     show(np.array([r,g,b]))
    # elif sensor == "s2":
    #     r = utils.pct_clip(img.read(4))
    #     g = utils.pct_clip(img.read(3))
    #     b = utils.pct_clip(img.read(2))
    #     show(np.array([r,g,b]))
    # elif sensor == "s1s2":
    #     r = utils.pct_clip(img.read(4))
    #     g = utils.pct_clip(img.read(3))
    #     b = utils.pct_clip(img.read(14))
    #     show(np.array([r,g,b]))

    # hent ut crs og lag en transform for å kunne skrive til tiff.
    img_profile = img.profile
    crs = img_profile["crs"]
    geo_transform = img_profile['transform']

    img_arr = img.read()
    B,H,W = img_arr.shape

    # mean,std = img_stats(img_arr, B)
    # transform = T.Normalize(mean=mean,std=std)
    # img_arr = utils.normalize(transform(torch.nan_to_num(
    #                       torch.from_numpy(img_arr.astype("float")).to(torch.float)
    #                       )))

    n_vert_chips = H//int(chip_size/2)-1
    n_hor_chips = W//int(chip_size/2)-1

    detection_raster = np.empty([4,H,W]).astype("float")
    detection_raster[:] = np.nan
    print("running inference ...")
    for i in tqdm(range(0,n_vert_chips)):
        for j in range(0,n_hor_chips):
            from_vert = int(chip_size/2*i)
            to_vert = from_vert + chip_size
            from_hor = int(chip_size/2*j)
            to_hor = from_hor + chip_size

            # chip = img_arr[:,from_vert:to_vert, from_hor:to_hor]
            chip = torch.from_numpy(img_arr[:,from_vert:to_vert, from_hor:to_hor].astype("float")).to(torch.float)

            with torch.no_grad():
                out = net(chip.unsqueeze(0).to("cuda"))

                score = torch.softmax(out,dim=1)

                _, prediction = torch.max(score,dim=1)
                score_background,     prediction_0 = torch.max(score[:, [0], :, :],dim=1)
                score_built_up,       prediction_1 = torch.max(score[:, [1], :, :],dim=1)
                score_roads,          prediction_2 = torch.max(score[:, [2], :, :],dim=1)
                score_infrastructure, prediction_3 = torch.max(score[:, 1:, :, :],dim=1)
            
            score = np.array([score_background.cpu().numpy(),
                                score_built_up.cpu().numpy(),
                                score_roads.cpu().numpy(),
                                score_infrastructure.cpu().numpy() #,
                                # prediction.cpu().numpy()
                                ]).squeeze()

            detection_raster[:,from_vert+buffer:to_vert-buffer, from_hor+buffer:to_hor-buffer] = np.nanmean(np.array([detection_raster[:,from_vert+buffer:to_vert-buffer, from_hor+buffer:to_hor-buffer],
                                                                                                                        score[:,buffer:-buffer,buffer:-buffer]]),
                                                                                                                        axis=0)


    output_path = os.path.join(archive, image_name + "_segmscore.tiff")
    print("writing image to disk ...")
    with rasterio.open(output_path,
                    mode='w',
                    driver='GTiff',
                    count=4,
                    width=W,
                    height=H,
                    crs=crs,
                    transform=geo_transform,
                    dtype=detection_raster.dtype) as outfile:
        outfile.write(detection_raster)
    print(output_path)
    break